In [1]:
import os, sys, json

autocog_home = os.path.realpath(f"{os.getcwd()}/../")
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.sta.syntax import Syntax

from autocog.utility.pynb import wrap_graphviz, display

In [2]:
from autocog.lm import RLM
from autocog.lm import Llama

models_path = "/data/models"
# model_name = None # Use random generator for ultra fast regression testing
# model_name = 'llama-2-7b.Q4_K_M'                 # wget https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf
# model_name = 'llama-2-7b-chat.Q4_K_M'            # wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
# model_name = 'tinyllama-1.1b-chat-v0.3.Q4_K_M'   # wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf
model_name = 'tinyllama-2-1b-miniguanaco.Q4_K_M' # wget https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf
n_ctx = 4096

if model_name is None:
    lm = RLM()
    syntax = Syntax()
else:
    lm = Llama(model_path=f"{models_path}/{model_name}.gguf", n_ctx=n_ctx)
    if model_name.find('llama-2') >= 0 and model_name.find('chat') >= 0:
        syntax = Syntax.Llama2Chat()
    if model_name.find('tinyllama') >= 0 and model_name.find('chat') >= 0:
        syntax = Syntax.ChatML()
    if model_name.find('tinyllama') >= 0 and model_name.find('miniguanaco') >= 0:
        syntax = Syntax.Guanaco()
    else:
        syntax = Syntax()


In [3]:
# r = [ lm.tokenize('the weather sucks', whole=False), [lm.model.token_nl()], lm.tokenize('next:', whole=False), lm.tokenize('\nnext:') ]
# print(r)
# print(lm.tokenize('>'))
# print(lm.tokenize('the weather sucks\nnext:'))
# # lm.model.token_nl()
# lm.detokenize([lm.model.token_nl()]+r[0]+r[1]+r[2])

In [4]:
# tokens = lm.tokenize('16', whole=False)
# lm.detokenize(tokens, whole=False)

In [5]:
# [].endswith()

In [6]:
arch = CogArch(lm=lm, syntax=syntax)

mmlu_cogs = [
    arch.load('mmlu-repeat',                filepath=f"{autocog_home}/library/mmlu-exams/repeat.sta"),
    arch.load('mmlu-repeat-cot',            filepath=f"{autocog_home}/library/mmlu-exams/repeat-cot.sta"),
    arch.load('mmlu-repeat-hyp',            filepath=f"{autocog_home}/library/mmlu-exams/repeat-hyp.sta"),
    arch.load('mmlu-repeat-iter',           filepath=f"{autocog_home}/library/mmlu-exams/repeat-iter.sta"),
    arch.load('mmlu-repeat-annot',          filepath=f"{autocog_home}/library/mmlu-exams/repeat-annot.sta"),
    arch.load('mmlu-select',                filepath=f"{autocog_home}/library/mmlu-exams/select.sta"),
    arch.load('mmlu-select-cot',            filepath=f"{autocog_home}/library/mmlu-exams/select-cot.sta"),
    arch.load('mmlu-select-hyp',            filepath=f"{autocog_home}/library/mmlu-exams/select-hyp.sta"),
    arch.load('mmlu-select-iter',           filepath=f"{autocog_home}/library/mmlu-exams/select-iter.sta"),
    arch.load('mmlu-select-annot',          filepath=f"{autocog_home}/library/mmlu-exams/select-annot.sta")
]

mmlu_data = [
    {
        "topic"    : "arithmetic",
        "question" : "What is 3*4+9?",
        "choices"  : [ "16", "21", "39", "42" ]
    }
]

arith_cogs = [
    arch.load('arithmetic-multiply-single', filepath=f"{autocog_home}/library/arithmetic/multiply-single.sta"),
    arch.load('arithmetic-multiply-chain',  filepath=f"{autocog_home}/library/arithmetic/multiply-chain.sta")
]

arith_data = [
    {
        "problem" : "What is the product of 392 and 42?"
    }
]

In [7]:
# TODO node prefixed by prompt name
# dotstr = '\n'.join([ prompt.toGraphViz_concrete() for prompt in arith_cogs[0].prompts.values() ])
# dotstr = arith_cogs[0].prompts['main'].instantiate(syntax=arch.syntax, stacks={}, branches={}, inputs=arith_data[0]).toGraphViz()
# display(wrap_graphviz(dotstr))

In [ ]:
tags = [
    'mmlu-repeat', 'mmlu-repeat-cot', 'mmlu-repeat-hyp', 'mmlu-repeat-iter', 'mmlu-repeat-annot',
    'mmlu-select', 'mmlu-select-cot', 'mmlu-select-hyp', 'mmlu-select-iter', 'mmlu-select-annot'
]
results = [ await arch(tag, **data) for tag in tags for data in mmlu_data ]
# arith_results = [ await cog(**data) for cog in arith_cogs for data in arith_data ]
results

In [ ]:
import numpy
scoring = lambda probas: numpy.power(numpy.prod(probas), 1./len(probas))
texts = arch.orchestrator.pages[-1].ftts['main'][-1].results(lm)
for text in texts:
    print(f"p={text[1]}\n{text[0]}")

In [ ]:
arch.orchestrator.pages[-1]